In [134]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [135]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

from functions import *

In [171]:
def get_ratio_dataframe(filepath):

    df_ratio = pd.read_csv(filepath,index_col=0)
    df_ratio = df_ratio.loc['2015-01-01':'2022-01-31']

    df_ratio = df_ratio.replace([np.inf, -np.inf], np.nan)
    df_ratio = df_ratio.apply(lambda row: row.fillna(row.mean()), axis=1)  # fill possible nan/inf with average of the row

    return df_ratio 

In [144]:
df_debt_to_market_cap = pd.read_csv('debt_to_market_cap_325.csv',index_col=0)
df_return_on_investment = pd.read_csv('return_on_investment_325.csv',index_col=0)
df_price_to_earnings = pd.read_csv('price_to_earnings_325.csv',index_col=0)

In [145]:
df_debt_to_market_cap = df_debt_to_market_cap.loc['2015-01-01':'2022-01-31']
df_return_on_investment = df_return_on_investment.loc['2015-01-01':'2022-01-31']
df_price_to_earnings = df_price_to_earnings.loc['2015-01-01':'2022-01-31']

print(len(df_debt_to_market_cap))
print(len(df_return_on_investment))
print(len(df_price_to_earnings))

1791
1791
1791


In [149]:
# fill possible nan with average of the row

df_debt_to_market_cap = df_debt_to_market_cap.apply(lambda row: row.fillna(row.mean()), axis=1)
df_return_on_investment = df_return_on_investment.apply(lambda row: row.fillna(row.mean()), axis=1)
df_price_to_earnings = df_price_to_earnings.apply(lambda row: row.fillna(row.mean()), axis=1)

In [160]:
df_debt_to_market_cap

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,1.004307,0.332626,0.966785,0.938788,1.729327,0.491293,0.267429,0.969831,2.871560,0.954448,...,1.261020,1.635962,1.223843,0.291060,0.977818,1.314769,0.450004,0.598621,0.724174,0.228452
2015-01-05,1.042413,0.332552,0.985852,0.952444,1.769732,0.513809,0.277014,0.984318,2.955700,0.990570,...,1.317497,1.652568,1.274285,0.298655,0.987341,1.379081,0.453145,0.611974,0.726741,0.220236
2015-01-06,1.065014,0.336371,1.011671,0.967666,1.798683,0.518996,0.282585,0.978956,3.023856,1.013812,...,1.345669,1.651278,1.291834,0.304415,1.011620,1.422282,0.458839,0.620275,0.733895,0.222119
2015-01-07,1.046225,0.333666,1.020173,0.928723,1.791122,0.508552,0.278400,0.965254,3.019215,1.017982,...,1.322318,1.649991,1.298807,0.306710,1.001272,1.418819,0.447248,0.612429,0.713464,0.216729
2015-01-08,1.008682,0.326946,0.977910,0.911731,1.793006,0.493243,0.282528,0.956998,2.973573,1.006688,...,1.284714,1.631234,1.286429,0.303557,0.976137,1.388392,0.436472,0.596021,0.701863,0.214452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,0.944680,0.505183,1.471253,0.704387,0.846795,0.733519,0.354643,1.488162,3.553969,0.728783,...,1.906741,1.828866,1.416744,0.127980,1.786782,0.625147,0.450569,0.797358,0.841133,0.712855
2022-01-26,0.957044,0.518646,1.487460,0.711346,0.856779,0.724458,0.335505,1.490340,3.514340,0.730919,...,1.926580,1.829484,1.405103,0.130239,1.825267,0.637576,0.452371,0.792204,0.839284,0.715576
2022-01-27,0.970755,0.517053,1.530548,0.736386,0.854922,0.738135,0.326778,1.480838,3.570073,0.750891,...,1.909933,1.850875,1.397448,0.149720,1.825969,0.638262,0.450228,0.779795,0.858728,0.711150


In [155]:
df_debt_to_market_cap1 = df_debt_to_market_cap.assign(**df_debt_to_market_cap.iloc[:, :].rank(axis = 1, ascending = True).astype(int))
df_debt_to_market_cap1

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,213,41,207,202,275,88,20,208,308,205,...,245,272,240,25,211,252,76,118,144,13
2015-01-05,216,38,206,200,274,94,20,205,308,209,...,247,270,242,28,208,255,74,119,141,13
2015-01-06,217,38,209,200,274,93,21,201,308,210,...,248,270,240,30,208,258,73,119,142,13
2015-01-07,217,38,213,194,275,91,21,202,308,212,...,246,269,241,30,209,257,71,120,138,13
2015-01-08,215,39,208,194,277,89,24,205,308,214,...,242,270,243,30,207,258,70,118,137,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,160,68,233,109,144,121,33,234,300,117,...,264,260,224,1,255,84,55,135,141,111
2022-01-26,159,69,231,109,146,113,22,232,299,119,...,263,258,220,1,257,88,54,131,141,111
2022-01-27,160,69,235,114,141,116,18,231,299,121,...,264,261,218,1,257,84,51,128,142,106


In [161]:
df_return_on_investment

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.644250,1.517516,1.641896,1.108146,1.082907,2.251344,2.853937,1.252275,1.992548,1.400091,...,6.786871,1.743871,4.967058,1.638539,2.472775,-4.411265,1.800576,1.357458,1.432706,1.934236
2015-01-05,0.663473,1.517195,1.675123,1.122582,1.107829,2.361059,2.960027,1.271246,2.050216,1.454613,...,7.049361,1.761592,5.143491,1.681397,2.497312,-4.647781,1.813293,1.388408,1.437784,1.864682
2015-01-06,0.674734,1.533708,1.720171,1.138623,1.125675,2.386417,3.021809,1.264224,2.096900,1.489756,...,7.179151,1.760216,5.204424,1.713898,2.559907,-4.807854,1.836350,1.407665,1.451939,1.880622
2015-01-07,0.665380,1.522015,1.735017,1.097478,1.121015,2.335387,2.975389,1.246282,2.093721,1.496067,...,7.071623,1.758843,5.228571,1.726847,2.533220,-4.794986,1.789421,1.389464,1.411515,1.834988
2015-01-08,0.646472,1.492928,1.661280,1.079415,1.122176,2.260823,3.021172,1.235477,2.062461,1.478980,...,6.897368,1.738825,5.185682,1.709056,2.468444,-4.682198,1.745820,1.351433,1.388561,1.815713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,2.923971,3.835106,1.114284,2.313102,0.581001,1.956053,1.553908,1.329892,2.045921,1.222930,...,1.308386,2.762759,0.445341,3.457550,-0.066529,20.268058,1.324025,0.978544,1.625547,1.026690
2022-01-26,2.966985,3.937337,1.126480,2.348851,0.588026,1.931885,1.471665,1.331700,2.022541,1.226661,...,1.321557,2.763705,0.441683,3.519049,-0.067967,20.906218,1.329453,0.971976,1.622023,1.030689
2022-01-27,3.014850,3.925239,1.158896,2.480835,0.586720,1.968366,1.434103,1.323811,2.055425,1.261568,...,1.310507,2.796450,0.439278,4.050232,-0.067993,20.941860,1.322996,0.956178,1.659062,1.024185


In [156]:
df_return_on_investment1 = df_return_on_investment.assign(**df_return_on_investment.iloc[:, :].rank(axis = 1, ascending = False).astype(int))
df_return_on_investment1

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,258,178,166,224,225,117,78,202,137,186,...,7,159,20,167,98,323,152,189,184,141
2015-01-05,257,180,167,224,225,111,75,202,134,185,...,7,161,20,165,100,323,155,191,186,148
2015-01-06,257,179,165,223,226,112,75,205,131,184,...,6,163,20,166,99,323,156,190,186,149
2015-01-07,257,179,162,226,223,114,75,207,132,181,...,6,160,19,165,100,323,159,190,187,153
2015-01-08,257,179,165,227,223,115,72,204,131,181,...,7,159,19,162,103,323,158,191,187,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,90,54,214,124,252,150,176,193,146,202,...,195,96,259,70,276,1,194,225,171,223
2022-01-26,90,51,214,125,253,152,182,193,148,202,...,195,99,259,68,276,1,194,226,173,223
2022-01-27,89,54,211,119,253,152,185,193,146,200,...,196,98,261,50,276,1,194,226,172,223


In [162]:
df_price_to_earnings

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,44.018182,106.778880,25.874089,139.488372,140.785714,46.928060,36.409194,45.895088,15.550752,100.729803,...,52.518325,75.741063,12.378770,90.942452,45.823289,24822.050885,93.877236,31.631135,84.348636,104.648069
2015-01-05,42.409091,106.802661,25.373680,137.488372,137.571429,44.871575,35.149361,45.219608,15.108066,97.056626,...,50.267016,74.979993,11.888766,88.629498,45.381314,23664.497553,93.226441,30.941001,84.050716,108.551812
2015-01-06,41.509091,105.589805,24.726092,135.325581,135.357143,44.423168,34.456452,45.467284,14.767539,94.831529,...,49.214660,75.038537,11.727256,86.952606,44.292163,22945.694275,92.069471,30.526920,83.231436,107.631647
2015-01-07,42.254545,106.445939,24.520041,141.000000,135.928571,45.335443,34.974384,46.112742,14.790240,94.443020,...,50.083770,75.097081,11.664294,86.302088,44.749922,23001.704920,94.455721,30.917996,85.614797,110.308493
2015-01-08,43.827273,108.633836,25.579731,143.627907,135.785714,46.742512,34.463451,46.510525,15.017259,95.502590,...,51.549738,75.960602,11.776530,87.198358,45.902213,23505.800726,96.787738,31.769161,87.029917,111.479616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,19.994885,103.530427,37.635101,72.821918,271.750000,102.444106,72.162405,54.442583,44.926253,87.932987,...,139.766355,113.812011,211.788795,125.252401,inf,2.699650,136.787018,39.712221,123.730840,173.851332
2022-01-26,19.736573,100.842852,37.225033,72.109589,268.583333,103.725420,76.278733,54.363024,45.432864,87.675927,...,138.327103,113.773564,213.543345,123.080394,inf,2.647022,136.242153,39.970609,124.003435,173.190137
2022-01-27,19.457801,101.153603,36.177082,69.657534,269.166667,101.803449,78.315788,54.711858,44.723609,85.344022,...,139.532710,112.458663,214.713045,107.065463,inf,2.644177,136.890802,40.606641,121.195707,174.268173


In [168]:
df_price_to_earnings2 = df_price_to_earnings.replace([np.inf, -np.inf], np.nan)

In [169]:
df_price_to_earnings2

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,44.018182,106.778880,25.874089,139.488372,140.785714,46.928060,36.409194,45.895088,15.550752,100.729803,...,52.518325,75.741063,12.378770,90.942452,45.823289,24822.050885,93.877236,31.631135,84.348636,104.648069
2015-01-05,42.409091,106.802661,25.373680,137.488372,137.571429,44.871575,35.149361,45.219608,15.108066,97.056626,...,50.267016,74.979993,11.888766,88.629498,45.381314,23664.497553,93.226441,30.941001,84.050716,108.551812
2015-01-06,41.509091,105.589805,24.726092,135.325581,135.357143,44.423168,34.456452,45.467284,14.767539,94.831529,...,49.214660,75.038537,11.727256,86.952606,44.292163,22945.694275,92.069471,30.526920,83.231436,107.631647
2015-01-07,42.254545,106.445939,24.520041,141.000000,135.928571,45.335443,34.974384,46.112742,14.790240,94.443020,...,50.083770,75.097081,11.664294,86.302088,44.749922,23001.704920,94.455721,30.917996,85.614797,110.308493
2015-01-08,43.827273,108.633836,25.579731,143.627907,135.785714,46.742512,34.463451,46.510525,15.017259,95.502590,...,51.549738,75.960602,11.776530,87.198358,45.902213,23505.800726,96.787738,31.769161,87.029917,111.479616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,19.994885,103.530427,37.635101,72.821918,271.750000,102.444106,72.162405,54.442583,44.926253,87.932987,...,139.766355,113.812011,211.788795,125.252401,NaN,2.699650,136.787018,39.712221,123.730840,173.851332
2022-01-26,19.736573,100.842852,37.225033,72.109589,268.583333,103.725420,76.278733,54.363024,45.432864,87.675927,...,138.327103,113.773564,213.543345,123.080394,NaN,2.647022,136.242153,39.970609,124.003435,173.190137
2022-01-27,19.457801,101.153603,36.177082,69.657534,269.166667,101.803449,78.315788,54.711858,44.723609,85.344022,...,139.532710,112.458663,214.713045,107.065463,NaN,2.644177,136.890802,40.606641,121.195707,174.268173


In [170]:
df_price_to_earnings2.dropna(axis='columns')

,AAWW,ABT,ACCO,ACHC,ACM,ADM,AEP,AES,AIMC,AIN,...,WES,WEX,WM,WMB,WTS,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,44.018182,106.778880,25.874089,139.488372,46.928060,36.409194,45.895088,15.550752,100.729803,93.128431,...,134.887030,52.518325,75.741063,12.378770,90.942452,24822.050885,93.877236,31.631135,84.348636,104.648069
2015-01-05,42.409091,106.802661,25.373680,137.488372,44.871575,35.149361,45.219608,15.108066,97.056626,92.762360,...,128.550199,50.267016,74.979993,11.888766,88.629498,23664.497553,93.226441,30.941001,84.050716,108.551812
2015-01-06,41.509091,105.589805,24.726092,135.325581,44.423168,34.456452,45.467284,14.767539,94.831529,88.711176,...,126.885980,49.214660,75.038537,11.727256,86.952606,22945.694275,92.069471,30.526920,83.231436,107.631647
2015-01-07,42.254545,106.445939,24.520041,141.000000,45.335443,34.974384,46.112742,14.790240,94.443020,87.808201,...,124.069611,50.083770,75.097081,11.664294,86.302088,23001.704920,94.455721,30.917996,85.614797,110.308493
2015-01-08,43.827273,108.633836,25.579731,143.627907,46.742512,34.463451,46.510525,15.017259,95.502590,88.174272,...,128.123476,51.549738,75.960602,11.776530,87.198358,23505.800726,96.787738,31.769161,87.029917,111.479616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,19.994885,103.530427,37.635101,72.821918,102.444106,72.162405,54.442583,44.926253,87.932987,86.900884,...,36.094659,139.766355,113.812011,211.788795,125.252401,2.699650,136.787018,39.712221,123.730840,173.851332
2022-01-26,19.736573,100.842852,37.225033,72.109589,103.725420,76.278733,54.363024,45.432864,87.675927,85.587521,...,36.308599,138.327103,113.773564,213.543345,123.080394,2.647022,136.242153,39.970609,124.003435,173.190137
2022-01-27,19.457801,101.153603,36.177082,69.657534,101.803449,78.315788,54.711858,44.723609,85.344022,84.263734,...,36.812885,139.532710,112.458663,214.713045,107.065463,2.644177,136.890802,40.606641,121.195707,174.268173


In [163]:
df_price_to_earnings.loc['2022-01-31']['WWW']

inf

In [158]:
df_price_to_earnings1 = df_price_to_earnings.assign(**df_price_to_earnings.iloc[:, :].rank(axis = 1, ascending = True).astype(int))
df_price_to_earnings1

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,72,226,19,252,253,84,54,80,8,217,...,105,176,4,204,79,314,209,37,189,223
2015-01-05,69,232,19,252,253,80,54,82,8,216,...,98,178,4,204,83,314,209,38,191,234
2015-01-06,68,231,19,251,252,80,54,84,8,216,...,97,179,4,203,79,313,211,38,192,235
2015-01-07,71,228,19,253,250,81,52,85,8,213,...,97,178,4,199,79,313,214,38,195,234
2015-01-08,70,230,19,253,249,83,47,82,8,213,...,100,174,4,197,80,313,214,38,195,233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,13,201,63,138,257,198,136,103,80,170,...,229,216,249,222,324,1,228,67,221,241
2022-01-26,13,198,64,138,258,202,147,102,81,173,...,230,215,249,221,324,1,228,70,222,242
2022-01-27,13,199,59,134,257,201,151,107,81,169,...,230,215,250,209,324,1,229,72,221,242


In [176]:
df_price_to_earnings11 = get_ratio_dataframe('price_to_earnings_325.csv')
df_price_to_earnings11

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,44.018182,106.778880,25.874089,139.488372,140.785714,46.928060,36.409194,45.895088,15.550752,100.729803,...,52.518325,75.741063,12.378770,90.942452,45.823289,24822.050885,93.877236,31.631135,84.348636,104.648069
2015-01-05,42.409091,106.802661,25.373680,137.488372,137.571429,44.871575,35.149361,45.219608,15.108066,97.056626,...,50.267016,74.979993,11.888766,88.629498,45.381314,23664.497553,93.226441,30.941001,84.050716,108.551812
2015-01-06,41.509091,105.589805,24.726092,135.325581,135.357143,44.423168,34.456452,45.467284,14.767539,94.831529,...,49.214660,75.038537,11.727256,86.952606,44.292163,22945.694275,92.069471,30.526920,83.231436,107.631647
2015-01-07,42.254545,106.445939,24.520041,141.000000,135.928571,45.335443,34.974384,46.112742,14.790240,94.443020,...,50.083770,75.097081,11.664294,86.302088,44.749922,23001.704920,94.455721,30.917996,85.614797,110.308493
2015-01-08,43.827273,108.633836,25.579731,143.627907,135.785714,46.742512,34.463451,46.510525,15.017259,95.502590,...,51.549738,75.960602,11.776530,87.198358,45.902213,23505.800726,96.787738,31.769161,87.029917,111.479616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,19.994885,103.530427,37.635101,72.821918,271.750000,102.444106,72.162405,54.442583,44.926253,87.932987,...,139.766355,113.812011,211.788795,125.252401,6187.724390,2.699650,136.787018,39.712221,123.730840,173.851332
2022-01-26,19.736573,100.842852,37.225033,72.109589,268.583333,103.725420,76.278733,54.363024,45.432864,87.675927,...,138.327103,113.773564,213.543345,123.080394,6133.028723,2.647022,136.242153,39.970609,124.003435,173.190137
2022-01-27,19.457801,101.153603,36.177082,69.657534,269.166667,101.803449,78.315788,54.711858,44.723609,85.344022,...,139.532710,112.458663,214.713045,107.065463,6066.388290,2.644177,136.890802,40.606641,121.195707,174.268173


In [175]:
df_price_to_earnings

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,44.018182,106.778880,25.874089,139.488372,140.785714,46.928060,36.409194,45.895088,15.550752,100.729803,...,52.518325,75.741063,12.378770,90.942452,45.823289,24822.050885,93.877236,31.631135,84.348636,104.648069
2015-01-05,42.409091,106.802661,25.373680,137.488372,137.571429,44.871575,35.149361,45.219608,15.108066,97.056626,...,50.267016,74.979993,11.888766,88.629498,45.381314,23664.497553,93.226441,30.941001,84.050716,108.551812
2015-01-06,41.509091,105.589805,24.726092,135.325581,135.357143,44.423168,34.456452,45.467284,14.767539,94.831529,...,49.214660,75.038537,11.727256,86.952606,44.292163,22945.694275,92.069471,30.526920,83.231436,107.631647
2015-01-07,42.254545,106.445939,24.520041,141.000000,135.928571,45.335443,34.974384,46.112742,14.790240,94.443020,...,50.083770,75.097081,11.664294,86.302088,44.749922,23001.704920,94.455721,30.917996,85.614797,110.308493
2015-01-08,43.827273,108.633836,25.579731,143.627907,135.785714,46.742512,34.463451,46.510525,15.017259,95.502590,...,51.549738,75.960602,11.776530,87.198358,45.902213,23505.800726,96.787738,31.769161,87.029917,111.479616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,19.994885,103.530427,37.635101,72.821918,271.750000,102.444106,72.162405,54.442583,44.926253,87.932987,...,139.766355,113.812011,211.788795,125.252401,NaN,2.699650,136.787018,39.712221,123.730840,173.851332
2022-01-26,19.736573,100.842852,37.225033,72.109589,268.583333,103.725420,76.278733,54.363024,45.432864,87.675927,...,138.327103,113.773564,213.543345,123.080394,NaN,2.647022,136.242153,39.970609,124.003435,173.190137
2022-01-27,19.457801,101.153603,36.177082,69.657534,269.166667,101.803449,78.315788,54.711858,44.723609,85.344022,...,139.532710,112.458663,214.713045,107.065463,NaN,2.644177,136.890802,40.606641,121.195707,174.268173


In [177]:
df_price_to_earnings11.assign(**df_price_to_earnings11.iloc[:, :].rank(axis = 1, ascending = True).astype(int))

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,72,226,19,252,253,84,54,80,8,217,...,105,176,4,204,79,314,209,37,189,223
2015-01-05,69,232,19,252,253,80,54,82,8,216,...,98,178,4,204,83,314,209,38,191,234
2015-01-06,68,231,19,251,252,80,54,84,8,216,...,97,179,4,203,79,313,211,38,192,235
2015-01-07,71,228,19,253,250,81,52,85,8,213,...,97,178,4,199,79,313,214,38,195,234
2015-01-08,70,230,19,253,249,83,47,82,8,213,...,100,174,4,197,80,313,214,38,195,233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,13,201,63,138,257,198,136,103,80,170,...,229,216,249,222,282,1,228,67,221,241
2022-01-26,13,198,64,138,258,202,147,102,81,173,...,230,215,249,221,282,1,228,70,222,242
2022-01-27,13,199,59,134,257,201,151,107,81,169,...,230,215,250,209,282,1,229,72,221,242


In [178]:
df_price_to_earnings1

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,ADM,AEP,AES,AIMC,...,WEX,WM,WMB,WTS,WWW,X,XRAY,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,72,226,19,252,253,84,54,80,8,217,...,105,176,4,204,79,314,209,37,189,223
2015-01-05,69,232,19,252,253,80,54,82,8,216,...,98,178,4,204,83,314,209,38,191,234
2015-01-06,68,231,19,251,252,80,54,84,8,216,...,97,179,4,203,79,313,211,38,192,235
2015-01-07,71,228,19,253,250,81,52,85,8,213,...,97,178,4,199,79,313,214,38,195,234
2015-01-08,70,230,19,253,249,83,47,82,8,213,...,100,174,4,197,80,313,214,38,195,233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,13,201,63,138,257,198,136,103,80,170,...,229,216,249,222,324,1,228,67,221,241
2022-01-26,13,198,64,138,258,202,147,102,81,173,...,230,215,249,221,324,1,228,70,222,242
2022-01-27,13,199,59,134,257,201,151,107,81,169,...,230,215,250,209,324,1,229,72,221,242
